In [ ]:
# General Libararies
import time
import os, sys
import numpy as np
import cv2
from tqdm import tqdm
import random
import importlib
import seaborn as sns
import pandas as pd
import array as arr

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras

# Layers, loss, optimzer, model 
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import losses
from tensorflow.keras.optimizers import *
from tensorflow.keras import datasets, layers, models,losses
from keras.layers import Activation, Dense
from tensorflow.keras.models import Model

# sklearn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix,roc_auc_score,confusion_matrix
import sklearn.metrics as metrics

# Google Drive
from google.colab import files
from google.colab import drive

# Plotting 
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import plotly.graph_objects as go
from matplotlib import figure
from plotly.subplots import make_subplots
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

# image processing 
**Loading Libraries**
from skimage import io
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import load_img

# Imports PIL module 
from PIL import Image
# inception
#https://keras.io/api/applications/#usage-examples-for-image-classification-models
from keras.applications.inception_v3 import InceptionV3

#https://stackoverflow.com/questions/49471467/keras-implementation-of-inception-v3-does-not-have-the-bn-auxillary

#Add Callbacks, e.g. ModelCheckpoints, earlystopping, csvlogger.
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from pickle import TRUE

# to initialize the random number generator.
import random
np.random.seed(1)
random.seed(1)
tensorflow.random.set_seed(1)
tf.random.set_seed(1)
tf.keras.utils.set_random_seed(1)   
#tf.config.experimental.enable_op_determinism()
tf.keras.backend.set_image_data_format('channels_last')

# for Time claculiting 
from timeit import default_timer as timer
from psutil import virtual_memory

**GPU-Beschleuniger zu aktiviere/**
**Erweiterter RAM**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print(virtual_memory().total)
if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

**Mount Google Drive**

In [ ]:
drive.mount('/content/drive')



**load data**

In [ ]:
# https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset/download
! unzip /content/drive/MyDrive/Training_Neronal_Network/archive.zip
# list all images 
training_loc = 'Training/'
testing_loc  = 'Testing/'
notumor_loc=os.listdir('./Training/notumor')
pituitary_loc=os.listdir('./Training/pituitary')
glioma_loc=os.listdir('./Training/glioma')
meningioma_loc=os.listdir('./Training/meningioma')
target_labels = os.listdir(training_loc)

**Connect to Lib /my_models function**

In [ ]:
sys.path.append('../content/drive/MyDrive/Funktionen')
#sys.path.append('../content/drive/MyDrive/Funktionen/my.py') -->
import importlib  
from lib import *
from my_models import *
import lib 
import my_models 
#to update changing 
importlib.reload(lib) 
importlib.reload(my_models)

**Connect to Libfunction/my_models**
**Image generator**

In [ ]:
# return image generator 
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
#to know how to define the input_shape in  InceptionV3 (150, 150, 3)
training_loc_target_size=print_max_min_dim(training_loc) 
testing_locc_target_size=print_max_min_dim(testing_loc)
targetSize = (150, 150) #input_shape
target_labels = ["glioma", "meningioma", "notumor", "pituitary"]
# create image generator 
train_gen, val_gen = img_gen(training_loc,target_labels,targetSize = targetSize)
**Plot Data**
#to plot dat
image_folder= ['glioma', 'meningioma', 'notumor', 'pituitary']
plt.figure(1)
plot_data_distribution_training(image_folder,training_loc)
plt.figure(2)
plot_data_distribution_testing(image_folder,testing_loc)

**Build Inception V3 Model**

In [ ]:
# https://keras.io/examples/vision/image_classification_from_scratch/
#((kernel_size)*stride+1)*filters)
from keras.applications.inception_v3 import InceptionV3
def modell_erstellen():
    tf.keras.backend.set_image_data_format('channels_last')
    conv_InceptionV3 =   InceptionV3(include_top=False,weights='imagenet',
                                     input_shape=(150,150,3))# https://image-net.org/challenges/LSVRC/ https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8530098/
    for layer in conv_InceptionV3.layers:
              layer.trainable = False

    # Create a new 'top' of the model
    top_of_model = conv_InceptionV3.output
    top_of_model = Flatten(name="flatten")(top_of_model) # multidimensional output and plan to make it linear so it may be sent onto a Dense layer
    top_of_model = Dense(4096, activation='relu')(top_of_model)
    top_of_model = Dropout(0.1)(top_of_model) # reducing too much connection between characteristics by lowering the weights (nodes) at a probability  https://arxiv.org/pdf/1207.0580.pdf
    output_layer = Dense(4, activation='softmax')(top_of_model) # the last Layer 4 classes 

    # Set up a new layer at the top of the model that is fully interconnected.
    model = Model(inputs=conv_InceptionV3.input, outputs=output_layer)
    return model

**Hyperparameters Searching space**

In [ ]:
batch_sizes = [ 8, 16, 32, 64, 128] #  NUM_SAMPLES * SIZE_OF_SAMPLE.
decay_rate=[1,10,100,1000,10000]
learning_rate=0.000001
epochs_count = 100
beta_2=[0.99, 0.999, 0.9999]
# the models name 
name=['eins','zwei','drei','vier','funf']
decay_rate_name=['1e-06', '1e-05',' 0.0001', '0.001','0.01']
batch_sizes_name=['8' ,'16' ,'32' ,'64' ,'128']
beta_2_name=['0.99', '0.999', '0.9999']

**to make an new model with neu Wights**

In [ ]:
model = modell_erstellen()
model.save_weights('../content/drive/MyDrive/Inception/')

**Compile the Model/Train Model/Save model to Google Drive**
 + (1) to investigate the Parameter  learn rate changing
 + (2) to investigate the Parameter Batch Sizes changing 
 + (3) to investigate the Parameter Momentum β2 s changing

In [ ]:

#(1) to investigate the Parameter  learn rate changing
experement=[1,2,3]
batch_size=32
train_gen, val_gen = img_gen(training_loc,target_labels, targetSize, batch_size=batch_size)
for h in range (len(experement)):
  times = np.zeros((len(decay_rate), 2),dtype=object)
  k=0
  batch_size=32
  beta_1=0.9 
  beta_2=0.999
  for i in range(len(decay_rate)):
      # The first iteration of the loop will be lrate_changing = 0.000001*1
      model = modell_erstellen()
      model.load_weights('../content/drive/MyDrive/Inception/')
      lrate_changing = learning_rate * decay_rate[i]  
      # train_gen, val_gen = img_gen(training_loc,target_labels, targetSize, batch_size=batch_size)
      History_model_basic, t_end =model_compiler(model, val_gen, train_gen, lrate_changing, beta_1, beta_2, batch_size, epochs_count, name[i],experement[h]) 
      # to save the Time after Training 
      times[k, 0] = t_end # (1)
      times[k,1]=str('lrate_changing:'+str(float(lrate_changing))+'')  # (1)
      k=k+1
      del model

  # to save the training time after training prozess
  pd.DataFrame(times).to_csv("../content/drive/MyDrive/Hyperparameters"+str(experement[h])+"/Inception_lrate_changing.csv"

In [ ]:
# (2) to investigate the Parameter Batch Sizes changing
# when the Batch Sizes ∈ batch_sizes = [ 8, 16, 32, 64, 128] is changed  but still  β1, β2 , learning_rate are constant 
# This loop will run 5  times for number of hyperparameters
experement=[1,2,3]
batch_sizes=[ 8, 16, 32, 64, 128]
# to make a Matrix with same lengeth from batch_sizes , decay_rate or  β2 for to save the Training time 
for h in range (len(experement)):

  times = np.zeros((len(batch_sizes), 2),dtype=object)
  k=0
  lrate_changing =0.00001
  beta_1=0.9 
  beta_2=0.999
  for i in range(len(batch_sizes)):

      batch_size=batch_sizes[i]
      model = modell_erstellen()
      model.load_weights('../content/drive/MyDrive/Inception/')
      # be careful when selecting which parameters to examine (1),(2),(3) , re-comment the constante parameters learning_rate , Batch Size, β1 ,β2,lrate_changing
      # the below function give back the History of Model and the Training time 
      

      # train_gen, val_gen = img_gen(training_loc,target_labels, targetSize, batch_size=batch_size)
      History_model_basic, t_end =model_compiler(model, val_gen, train_gen, lrate_changing, beta_1, beta_2, batch_size, epochs_count, name[i],experement[h]) 

      # to save the Time after Training 
      times[k, 0] = t_end # (2)
      times[k,1]=str('batch_size:'+str(batch_sizes[i])+'') # (2) 
      k=k+1
      del model
  # to save the training time after  
  pd.DataFrame(times).to_csv("../content/drive/MyDrive/Hyperparameters"+str(experement[h])+"/Inception_batch_changing.csv")

In [ ]:
# (3) to investigate the Parameter Momentum β2 s changing
# when the β2 ∈ [0.99, 0.999, 0.9999] is changed but still  β1 , learning_rate , Batch Size are constant 
# This loop will run 3  times for number of hyperparameters

experement=[1,2]
for h in range (len(experement)):
  k=0
  beta_2=[0.99, 0.999, 0.9999]
  times = np.zeros((len(beta_2), 2),dtype=object)
  lrate_changing =0.00001
  batch_size =128
  for i in range(len(beta_2)):
      beta_1=0.9
      model = modell_erstellen()
      model.load_weights('../content/drive/MyDrive/Inception/')
      # be careful when selecting which parameters to examine (1),(2),(3) , re-comment the constante parameters learning_rate , Batch Size, β1 ,β2,lrate_changing
      # the below function give back the History of Model and the Training time

      History_model_basic,t_end=model_compiler(model,val_gen,train_gen,lrate_changing,beta_1, beta_2[i],batch_size,epochs_count,name[i],experement[h])

      # to save the Time after Training 
      times[k, 0] = t_end # (3)

      times[k,1]=str('beta_2:'+str(float(beta_2[i]))+'') # (3)

      k=k+1
      del model
  # to save the training time after  
  pd.DataFrame(times).to_csv("../content/drive/MyDrive/Hyperparameters"+str(experement[h])+"/Inception_beta_changing.csv")

**After Save Models/Load all**

In [ ]:
models_numbers= decay_rate# (1) decay_rate ,(2) batch_sizes or (3) beta_2  muss to be the same number of models 
json_h5_pathfile='../content/drive/MyDrive/Hyperparameters/'
speicher=loading_models(json_h5_pathfile,name,models_numbers)

In [ ]:
#this function to load all models from drive the first Parameter is the filepath the sekund is the name=['eins','zwei','drei','vier','funf']
#speicher here is a placeholder to save all models in it, all models will be uploaded
def loading_models(json_h5_pathfile,name,models_numbers):
  speicher= [None] * len(models_numbers)
  for i in range(len(models_numbers)):
    # load json and create model 

    json_file = open(''+json_h5_pathfile+'/model_'+name[i]+'.json','r')

    loaded_model_json= json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    speicher[i]=loaded_model

    # load weights into new model
    loaded_model.load_weights(""+json_h5_pathfile+"/model_"+name[i]+".h5")
    print("Loaded model from disk")
  return speicher


**Testing the Model**

In [ ]:
#def loading_models(json_h5_pathfile, name, models_numbers):
#data Aug for Testing
Generator_test = ImageDataGenerator(rescale = 1./255,shear_range=0.2)
test_gen      = Generator_test.flow_from_directory(testing_loc,
# resizing testing images to (min(heights), min(widths)) in both trainining and testing sets
target_size = (150,150),
color_mode = 'rgb',
classes = target_labels,
class_mode = 'categorical',
batch_size = 1,
shuffle = False,subset = None)

**Plot the accuracy loss val_accuracy val_loss**
 + (1) to investigate the Parameter  learn rate changing
 + (2) to investigate the Parameter Batch Sizes changing 
 + (3) to investigate the Parameter Momentum β2 s changing

In [ ]:
# (1) to investigate the Parameter  learn rate changing
# when the learn rate ∈ 0.000001* [1,10,100,1000,10000] is changed and  but still  β1, β2 ,  Batch Size are constant  

csv_pathfile_plot_acc_loss='../content/drive/MyDrive/Hyperparameters3/model_'
batch_sizes=32
decay_rates =[1,10,100,1000,10000]
plot_acc_loss_changed_learnrate(csv_pathfile_plot_acc_loss,name,batch_sizes, decay_rates)

json_h5_pathfile='../content/drive/MyDrive/Hyperparameters3/'

speicher = loading_models(json_h5_pathfile,name,decay_rates)
classification_report_testing_confusion_matrix_learnrate(json_h5_pathfile,speicher,name,batch_sizes,decay_rates,test_gen,target_labels)
csv_pathfile_time = pd.read_csv('/content/drive/MyDrive/Hyperparameters3/Inception_lrate_changing.csv')
save_loc='/content/drive/MyDrive/Hyperparameters3/'
changed_hyperparameter_name='learning rate'#
values_of_parameter=decay_rate_name 
plot_training_time(csv_pathfile_time ,save_loc,values_of_parameter,changed_hyperparameter_name)

In [ ]:
# (2) to investigate the Parameter Batch Sizes changing
json_h5_pathfile='../content/drive/MyDrive/Hyperparameters3/'
batch_sizes=[8,16,32, 64, 128]
learning_rate =0.00001
speicher = loading_models(json_h5_pathfile,name,batch_sizes)
classification_report_testing_confusion_matrix_batchsize(json_h5_pathfile,speicher,name,batch_sizes,learning_rate,test_gen,target_labels)
csv_pathfile_plot_acc_loss='/content/drive/MyDrive/Hyperparameters3/model_'

plot_acc_loss_changed_batchsize(csv_pathfile_plot_acc_loss,name,batch_sizes,learning_rate)
csv_pathfile_time = pd.read_csv('/content/drive/MyDrive/Hyperparameters3/Inception_batch_changing.csv')
save_loc='/content/drive/MyDrive/Hyperparameters3/'
changed_hyperparameter_name='Batch Size'#
values_of_parameter=batch_sizes_name
plot_training_time(csv_pathfile_time,save_loc,values_of_parameter,changed_hyperparameter_name)

In [ ]:
# (3) to investigate the Parameter Momentum β2 s changing
# when the β2 ∈ [0.99, 0.999, 0.9999] is changed but still  β1 , learning_rate , Batch Size are constant 
csv_pathfile_plot_acc_loss='../content/drive/MyDrive/Hyperparameters2/model_'
learning_rate=0.00001
batch_sizes=128
beta_1=0.9
beta_2=[0.99, 0.999, 0.9999]
plot_acc_loss_changed_beta_2(csv_pathfile_plot_acc_loss,name,batch_sizes,learning_rate,beta_1,beta_2)
json_h5_pathfile='../content/drive/MyDrive/Hyperparameters2/'
speicher =loading_models(json_h5_pathfile,name,beta_2)
classification_report_testing_confusion_matrix_beta_2(json_h5_pathfile,speicher,name,batch_sizes,learning_rate,beta_1,beta_2, test_gen,target_labels)
csv_pathfile_time = pd.read_csv('/content/drive/MyDrive/Saved_training1/InceptionV3/beta_2/Inception_beta_changing.csv')
save_loc='/content/drive/MyDrive/Saved_training1/InceptionV3/beta_2/'
changed_hyperparameter_name='Momentum β2'#
values_of_parameter=beta_2_name
plot_training_time(csv_pathfile_time,save_loc,values_of_parameter,changed_hyperparameter_name)

In [ ]:
experement=[1,2,3]
mittelwert=np.zeros((len(experement), 1),dtype=object)

csv_pathfile_plot_acc_loss='/content/drive/MyDrive/Saved_training1/Basic/lrate'
k=2   #model_type eins, zwei ..
for h in range (len(experement)):
  df = pd.read_csv(''+csv_pathfile_plot_acc_loss+''+str(experement[h])+'/model_'+name[k]+'.csv')
  dfx=df['epoch']
  dfval_loss=df['val_loss']
  mittelwert[h]=round(dfval_loss[dfx.iat[-1]],2)
print(mittelwert)